In [1]:
import numpy as np
import random
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.linalg import sqrtm
from skimage.transform import resize
import cv2
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.layers import Activation, Dense, Input, Dropout, BatchNormalization
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Reshape, Concatenate, Lambda, Layer
from keras.layers.merge import concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import mean_squared_error, binary_crossentropy
from keras import metrics
from keras.applications.inception_v3 import preprocess_input, InceptionV3
import keras.backend as K

Using TensorFlow backend.


In [3]:
## Constants
OLD_IMAGE_DIMS = (218, 178, 3)
NEW_IMAGE_DIMS = (64, 64, 3)
#CROP_IMAGE_DIMS = (25, 45, 153, 173)
BATCH_SIZE = 128
N = 10000
NUM_ATTRIBUTES = 40
NUM_BATCHES = 1
LATENT_DIM = 64
TOT_IMAGES = 202599
EPOCHS = 1

In [4]:
## Constants needed to run in Google Colab
PATH = './'
IMAGES = 'img_align_celeba/'
ATTRIBUTES = 'list_attr_celeba.txt'

In [5]:
## Loading dataset
def get_attributes(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    attributes = []
    attr = []
    i = 0
    for line in lines:
        if i != 0 and i != 1:
            line = line.split()
            #line.pop(0)
            attributes.append(line)
        i += 1
    for i in range(len(attributes)):
        for j in range(len(attributes[i])):
            if attributes[i][j] == '-1':
                attributes[i][j] = '0'
    return attributes

def initialize_training_set(): 
    all_attributes = get_attributes(PATH + ATTRIBUTES)
    
    while True:
        chosen_info = random.sample(all_attributes, N)
        chosen_attributes = [info[1:] for info in chosen_info]
        chosen_images = [info[0] for info in chosen_info]
        resized_images = []
    
        for ind in range(len(chosen_info)):
            image_path = PATH + IMAGES + chosen_images[ind]
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)[...,::-1] / 255.0
            img = img[45:173,25:153]
            img = cv2.resize(src=img, dsize=(64, 64))
            resized_images.append(img)

        resized_images = np.array(resized_images, dtype=np.float32)
        chosen_attributes = np.array(chosen_attributes, dtype=np.float32)

        yield [resized_images, chosen_attributes], None 
        
def initialize_test_set():
    all_attributes = get_attributes(PATH + ATTRIBUTES)
    info_in_batches = []
    attributes_in_batches = []
    images_in_batches = []
    test_set = []
    
    for i in range(NUM_BATCHES):
        chosen_info = random.sample(all_attributes, BATCH_SIZE)
        chosen_attributes = [info[1:] for info in chosen_info]
        chosen_images = [info[0] for info in chosen_info]
        
        info_in_batches.append(chosen_info)
        attributes_in_batches.append(chosen_attributes)
        images_in_batches.append(chosen_images)
        
    for i in range(NUM_BATCHES):
        chosen_images = images_in_batches[i]
        attributes = attributes_in_batches[i]
        resized_images = []
        
        for j in range(BATCH_SIZE):
            image_path = PATH + IMAGES + chosen_images[j]
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)[...,::-1] / 255.0
            img = img[45:173,25:153]
            img = cv2.resize(src=img, dsize=(64, 64))
            resized_images.append(img)

        resized_images = np.array(resized_images, dtype=np.float32)
        attributes = np.array(chosen_attributes, dtype=np.float32)

        test_set.append([resized_images, attributes])
    return test_set

training_set = initialize_training_set()
test_set = initialize_test_set()

In [6]:

class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = K.shape(z_mean)[0]
    dim = K.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def VAE():
  #Define encoder model.
  input_img = Input(shape = NEW_IMAGE_DIMS, name='input_img')
  labels = Input(shape = (NUM_ATTRIBUTES,), name='labels')

  x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(input_img)
  x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)
  x = Conv2D(filters = 128, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)
  x = Conv2D(filters = 256, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)
  x = Conv2D(filters = 512, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)

  #encode = Encode_Decode(input_img) #Done

  shape_before_flattening = K.int_shape(x)[1:]
  #shape_before_flattening = K.int_shape(encode)[1:]

  x = Flatten()(x)

  z_mean = Dense(LATENT_DIM, name='z_mean')(x)
  z_log_sigma = Dense(LATENT_DIM, name='z_log_sigma')(x)
  z = Sampling()([z_mean, z_log_sigma])

  zy = Concatenate()([z, labels])

  inputs_embedding = Input(shape=(LATENT_DIM + NUM_ATTRIBUTES,))
  embedding = Dense(np.prod(shape_before_flattening))(inputs_embedding)
  embedding = Reshape(shape_before_flattening)(embedding)

  #Decoding
  x_ = Conv2DTranspose(filters = 256, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(embedding)
  x_ = Conv2DTranspose(filters = 128, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(x_)
  x_ = Conv2DTranspose(filters = 64, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(x_)
  x_ = Conv2DTranspose(filters = 32, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(x_)
  x_ = Conv2DTranspose(filters = 3, kernel_size = 3, strides = 2,  padding = 'same', activation = 'sigmoid')(x_)

  #x_hat = Encode_Decode(embedding)

  encoder = Model(inputs = [input_img, labels], outputs = zy, name="encoder")
  decoder = Model(inputs = inputs_embedding, outputs = x_, name="decoder")

  vae_out = decoder(encoder([input_img, labels]))

  rec_loss =  np.prod(NEW_IMAGE_DIMS) * binary_crossentropy(Flatten()(input_img), Flatten()(vae_out))
  kl_loss = - 0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
  vae_loss = K.mean(rec_loss + kl_loss)

  vae = Model(inputs = [input_img, labels], outputs = vae_out, name="vae")

  vae.add_loss(vae_loss)

  optimizer = Adam(lr=0.0005, beta_1 = 0.5)
  vae.compile(optimizer)

  return vae, encoder, decoder

vae, encoder, decoder = VAE()

encoder.summary()
decoder.summary()



Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   896         input_img[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 16, 16, 64)   18496       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 8, 8, 128)    73856       conv2d_2[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


In [7]:
vae.fit(training_set, steps_per_epoch=TOT_IMAGES//BATCH_SIZE, verbose=1, epochs=EPOCHS)

Epoch 1/1


/usr/local/lib/python3.7/site-packages/keras/utils/data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


KeyboardInterrupt: 

In [ ]:
# Save model weights
vae.save_weights('./vae.h5')
decoder.save_weights('./decoder.h5')
encoder.save_weights('./encoder.h5')

In [ ]:
# Calculate Frechet Inception Distance (https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/)

# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


In [ ]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg')#, input_shape=(299,299,3))

In [ ]:
def reconstruct_images(test_set):
    print('Reconstructing images')
    images_for_fid = []
    predictions_for_fid = []
    
    index = 1
    
    for batch in test_set:
        print('Batch:', index)
        example_images = batch[0] #* 255.0
        example_predictions = vae.predict(batch) #* 255.0
        
        # Scaling images
        scaled_images = scale_images(example_images, (299, 299, 3))
        scaled_predictions = scale_images(example_predictions, (299, 299, 3))
        
        # Pre-processing images
        preprocessed_images = preprocess_input(scaled_images)
        preprocessed_predictions = preprocess_input(scaled_predictions)
        
        images_for_fid.append(preprocessed_images)
        predictions_for_fid.append(preprocessed_predictions)
                      
        index += 1

    return example_images, example_predictions, images_for_fid, predictions_for_fid


def generate_images():
    print('Generating images')
    
    predictions_for_fid = []
    
    generated_attributes = np.zeros(NUM_ATTRIBUTES, dtype=np.float32)
    generated_attributes[8] = 1   # black hair
    generated_attributes[31] = 1  # smiling
    
    images = np.random.uniform(-1, 1, size = (NUM_BATCHES * BATCH_SIZE, LATENT_DIM)).astype(np.float32)
    
    for i in range(NUM_BATCHES):
        print('Batch:', i+1)
        batch = []
        
        for j in range(BATCH_SIZE):
            img_info = []
            img_info.append(np.concatenate((images[j].flatten(), generated_attributes)))
            batch.append(np.array(img_info).flatten())
            
        batch = np.array(batch)
        predictions = decoder.predict(batch)
        
        # Scaling images
        scaled_predictions = scale_images(prediction, (299, 299, 3))
        
        # Pre-processing images
        preprocessed_predictions = preprocess_input(scaled_predictions)
        
        predictions_for_fid.append(preprocessed_predictions)

    return predictions, predictions_for_fid


In [ ]:
# Reconstructing images with VAE
rec_images, rec_predictions, images_for_fid, rec_predictions_for_fid = reconstruct_images(test_set) 

# Generating new images with VAE
gen_predictions, gen_predictions_for_fid = generate_images()

In [ ]:
def show_images(batch):
    used_indices = []
    nrow = 2
    ncol = 8
    fig = plt.figure(figsize=((ncol+1)*1.5, (nrow+1)*1.5)) 
    gs = mpl.gridspec.GridSpec(nrow, ncol,
             wspace=0.0, hspace=0.0, 
             top=1.-0.5/(nrow+1), bottom=0.5/(nrow+1), 
             left=0.5/(ncol+1), right=1-0.5/(ncol+1)) 
    for i in range(16):
        plt.subplot(nrow,ncol,i+1)
        plt.imshow(batch[i])
        plt.axis('off')
    plt.show()

In [ ]:
print('Original images')
show_images(rec_images)
print('\n')
print('Reconstructed images')
show_images(rec_predictions)

In [ ]:
fid_score = calculate_fid(model, images_for_fid, rec_predictions_for_fid)
print('FID score for reconstructed images:', fid_score)

In [ ]:
print('Generated images')
show_images(gen_predictions)

In [ ]:
fid_score = calculate_fid(model, images_for_fid, gen_predictions_for_fid)
print('FID score for reconstructed images:', fid_score)